### Define the follower arm, connect, read the position

In [1]:
from lerobot.common.robot_devices.motors.dynamixel import DynamixelMotorsBus
import numpy as np

follower_port = "/dev/ttyACM0"

follower_arm = DynamixelMotorsBus(
    port=follower_port,
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl430-w250"),
        "shoulder_lift": (2, "xl430-w250"),
        "elbow_flex": (3, "xl330-m288"),
        "wrist_flex": (4, "xl330-m288"),
        "wrist_roll": (5, "xl330-m288"),
        "gripper": (6, "xl330-m288"),
    },
)

follower_arm.connect()
follower_pos = follower_arm.read("Present_Position")

print(follower_pos)

[2051 3091 3055 3084 3096 1993]


### Turn on the torque

In [ ]:
from lerobot.common.robot_devices.motors.dynamixel import TorqueMode

follower_arm.write("Torque_Enable", TorqueMode.ENABLED.value)

In [ ]:
import time

# Get the current position
position = follower_arm.read("Present_Position")

# Update first motor (shoulder_pan) position by +10 steps
position[0] += 10
follower_arm.write("Goal_Position", position)

time.sleep(2.0)

# Update all motors position by 50 steps
position += 50
follower_arm.write("Goal_Position", position)

time.sleep(2.0)

# Update gripper by 50 steps
position[-1] += 50
follower_arm.write("Goal_Position", position[-1], "gripper")

time.sleep(2.0)


In [ ]:
follower_arm.write("Torque_Enable", TorqueMode.DISABLED.value)

### Do the Calibration

In [3]:
from lerobot.common.robot_devices.robots.utils import Robot

def safe_disconnect(func):
    # TODO(aliberts): Allow to pass custom exceptions
    # (e.g. ThreadServiceExit, KeyboardInterrupt, SystemExit, UnpluggedError, DynamixelCommError)
    def wrapper(robot, *args, **kwargs):
        try:
            return func(robot, *args, **kwargs)
        except Exception as e:
            if robot.is_connected:
                robot.disconnect()
            raise e

    return wrapper

@safe_disconnect
def calibrate(robot: Robot, arms: list[str] | None):
    # TODO(aliberts): move this code in robots' classes
    if robot.robot_type.startswith("stretch"):
        if not robot.is_connected:
            robot.connect()
        if not robot.is_homed():
            robot.home()
        return

    if arms is None:
        arms = robot.available_arms

    unknown_arms = [arm_id for arm_id in arms if arm_id not in robot.available_arms]
    available_arms_str = " ".join(robot.available_arms)
    unknown_arms_str = " ".join(unknown_arms)

    if arms is None or len(arms) == 0:
        raise ValueError(
            "No arm provided. Use `--arms` as argument with one or more available arms.\n"
            f"For instance, to recalibrate all arms add: `--arms {available_arms_str}`"
        )

    if len(unknown_arms) > 0:
        raise ValueError(
            f"Unknown arms provided ('{unknown_arms_str}'). Available arms are `{available_arms_str}`."
        )

    for arm_id in arms:
        arm_calib_path = robot.calibration_dir / f"{arm_id}.json"
        if arm_calib_path.exists():
            print(f"Removing '{arm_calib_path}'")
            arm_calib_path.unlink()
        else:
            print(f"Calibration file not found '{arm_calib_path}'")

    if robot.is_connected:
        robot.disconnect()

    # Calling `connect` automatically runs calibration
    # when the calibration file is missing
    robot.connect()
    robot.disconnect()
    print("Calibration is done! You can now teleoperate and record datasets!")

In [4]:
from lerobot.common.utils.utils import init_hydra_config, log_say
from lerobot.common.robot_devices.robots.factory import make_robot

# define the robot with YAML

robot_path = "lerobot/configs/robot/koch.yaml"
robot_cfg = init_hydra_config(robot_path)
robot = make_robot(robot_cfg)

calibrate(robot, ["main_follower"])

Removing '.cache/calibration/koch/main_follower.json'
Connecting main follower arm.
Missing calibration file '.cache/calibration/koch/main_follower.json'

Running calibration of koch main follower...

Move arm to zero position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_zero.webp

Move arm to rotated target position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_rotated.webp

Move arm to rest position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_rest.webp

Calibration is done! Saving calibration file '.cache/calibration/koch/main_follower.json'
Activating torque on main follower arm.
Calibration is done! You can now teleoperate and record datasets!


In [10]:
from lerobot.common.robot_devices.motors.dynamixel import TorqueMode
import time

robot.follower_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)

# if not robot.is_connected:
    # robot.connect()

play_sounds = True

all_poses = []
while True:
    robot.follower_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)

    voice = 'Please adjust the robot pose. Press "s" to turn on the stiffness. Press "q" to quit.'
    log_say(voice, play_sounds)
    command = input(voice)
    
    if command == "s":
        while True:
            robot.follower_arms["main"].write("Torque_Enable", TorqueMode.ENABLED.value)
        
            voice = 'If you want to save this pose, press "y" to save, "n" to re-adjust'
            log_say(voice, play_sounds)
            command = input(voice)

            if command == "y":
                curr_pos = robot.follower_arms["main"].read("Present_Position")
                all_poses.append(curr_pos)

                voice = "pose is saved. stiffness will be turned off after few seconds"
                log_say(voice, play_sounds)
                time.sleep(5)

                robot.follower_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)
                break

            elif command == "n":
                voice = "I will give you 10 seconds to re-adjust the pose. Please try after stiffness is off"
                log_say(voice, play_sounds)
                time.sleep(5)
                robot.follower_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)
                time.sleep(10)
            
            else:
                continue

    if command == "q":
        voice = "{} poses are saved. Please run below cell to replay. Stifeeness will be turn off after few seconds".format(len(all_poses))
        log_say(voice, play_sounds)
        break

time.sleep(5)
robot.follower_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)



In [37]:
import numpy as np

curr_pos = robot.follower_arms["main"].read("Present_Position")

final_poses = [curr_pos] + all_poses

stride = 80
stride_poses = []
for i in range(len(final_poses)-1):
    stride_poses.append(np.linspace(final_poses[i], final_poses[i+1], stride+1)[:-1])
stride_poses = np.concatenate(stride_poses, axis=0)


In [ ]:
robot.follower_arms["main"].write("Torque_Enable", TorqueMode.ENABLED.value)

time.sleep(5)
    
for i, p in enumerate(stride_poses):
    robot.follower_arms["main"].write("Goal_Position", p)
    if i == 1*stride:
        log_say("Hello! I can hold it for you", play_sounds)
    if i == 4*stride:
        log_say("HA. HA. HA", play_sounds)
    time.sleep(0.02)

time.sleep(5)

robot.follower_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)


In [23]:
robot.follower_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)
